In [ ]:
import sys
# setting path
sys.path.append('..')
# importing
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, IBMQ, BasicAer
from qiskit.aqua import QuantumInstance
import ECC_library
from qiskit.visualization import plot_bloch_multivector, plot_histogram

In [ ]:
def addition_without_carry(circuit,x,y,N):
    for i in range(1,N):
        circuit.cx(x[i],y[i])
    for i in range(N-2,0,-1):
        circuit.cx(x[i],x[i+1])
    for i in range(0,N-1):
        circuit.ccx(y[i],x[i],x[i+1])
    for i in range(N-1,0,-1):
        circuit.cx(x[i],y[i])
        circuit.ccx(y[i-1],x[i-1],x[i])
    for i in range(1,N-1):
        circuit.cx(x[i],x[i+1])
    for i in range(0,N):
        circuit.cx(x[i],y[i])

In [ ]:
"""check if b>a"""
def test_comparator(a,b,n):
    x = QuantumRegister(n)
    y = QuantumRegister(n)
    c = QuantumRegister(1)
    clas_a = ClassicalRegister(n)
    clas_b = ClassicalRegister(n)
    clas_c = ClassicalRegister(1)
    circuit = QuantumCircuit(x,y,c,clas_a,clas_b,clas_c)
    s_a=bin(int(a))[2:].zfill(n)
    s_b=bin(int(b))[2:].zfill(n)
    for i in range(0, n):
        if s_a[i] == '1':
            circuit.x(x[n-1-i])
    for i in range(0, n):
        if s_b[i] == '1':
            circuit.x(y[n-1-i])
    """if b<a, the sub gate will toggle the c qubit, leaving in the y register b-a mod 2**n"""
    ECC_library.subtraction(circuit,x,y,c,n)
    """After that, we perform an addition without affecting the c qubit"""
    addition_without_carry(circuit,x,y,n)
    for i in range(0, n):
        circuit.measure(x[i],clas_a[i])
    for i in range(0, n):
        circuit.measure(y[i],clas_b[i])
    circuit.measure(c,clas_c)
    print('a:',a,'b:',b)
    backend = BasicAer.get_backend('qasm_simulator')
    simulation = execute(circuit, backend=backend ,shots=10)
    sim_result = simulation.result()
    out=list(sim_result.get_counts().keys())[0]
    output=out.split(' ')
    output_a = output[2]
    output_b = output[1]
    output_c = output[0]
    res_a = int(output_a, 2)
    res_b = int(output_b, 2)
    res_c = int(output_c, 2)
    print('res_a:',res_a,'res_b:',res_b,'res_c:',res_c)
    if((b<a and res_c==0) or res_b!=b or (b>=a and res_c==1) or res_a!=a):
        print('found!')

In [ ]:
for i in range (0,2**5,3):
    for j in range (0,2**5,5):
        test_comparator(i,j,5)
        print('-----------')